In [ ]:
# Author: Malik Altakrori, PhD
# IBM Research
# malik.altakrori@ibm.com

In [ ]:
# This script is used to generate the .sh file 
# these .sh files are split for the different experiments 

# script is based on : https://github.com/primeqa/primeqa/blob/main/extensions/custom_mrc/run_mrc_eval.sh

In [ ]:
import os
import glob

In [ ]:
root_folder = "<Provide the abs path to the repo>"

# install primeqa from: https://github.com/primeqa/primeqa 
# notice that there are 2x primeQA. We want the 2nd level primeQA from the link above
primeqa_path = "<your_folder>/primeqa/primeqa" 

In [ ]:
# test run
def write_script(
    root_folder,
    setting, 
    lang,
    output_dir_parent, 
    fileNames,
    primeqa_path = "../primeqa/primeqa",
    model_name = "PrimeQA/nq_tydi_sq1-reader-xlmr_large-20221110",
    translated=False,
    new_file: bool = False):

    if len(fileNames) == 0:
        raise ValueError

    if not translated:
        script_path = os.path.join(root_folder, "Scripts", "original", f'run_mrc_eval_{lang}_{setting}.sh')
    else:
        script_path = os.path.join(root_folder, "Scripts", "translated", f'run_mrc_eval_{lang}_{setting}.sh')

    print(script_path)
    if new_file:
        with open(script_path, 'w') as shFile:
            # starting with the header
            shFile.write(
            """#!/usr/bin/env bash \ 
set -xeo pipefail \n\n""")
            
    for fname in fileNames:
        output_dir_child = os.path.basename(fname).split('.')[0]
        template = f"""python {os.path.join(primeqa_path, "primeqa/mrc/run_mrc.py")} \
--model_name_or_path {model_name} \
--eval_file {fname} \
--preprocessor primeqa.mrc.processors.preprocessors.squad.SQUADPreprocessor \
--postprocessor primeqa.mrc.processors.postprocessors.squad.SQUADPostProcessor \
--eval_metrics SQUAD \
--output_dir {os.path.join(root_folder, output_dir_parent, setting, f"Results_{lang}", output_dir_child)} \
--do_eval --per_device_eval_batch_size 8 \
--overwrite_output_dir --overwrite_cache"""
        with open(script_path, 'a+') as shFile:
            shFile.write(template)
            shFile.write("\n\n")

    print("done")

## Generate Script for experiments in Table 2 

In [ ]:
# input/output
script_location = "Settings"
output_dir_parent = "Results"

In [ ]:
overwrite_existing = True
for out_f in ['original', 'translated']:
    output_folder = os.path.join(root_folder, "Scripts", out_f)
    try:
        os.makedirs(output_folder, exist_ok=overwrite_existing)
    except:
        print(f"WARNING: Folder {output_folder} exists, either delete manually, or set overwrite_existing to True")
        raise ValueError

In [ ]:
"""
The experiments are used to compare two models on the SQuAD english dataset. So we compare with English only

"""
models = ["PrimeQA/nq_tydi-reader-xlmr_large-20221210", "PrimeQA/nq_tydi_sq1-reader-xlmr_large-20221110"]
with open(os.path.join(root_folder, "scripts", "original", f'run_mrc_eval_SQuAD.sh'), 'w') as shFile:
    # starting with the header
    shFile.write(
    """#!/usr/bin/env bash \ 
set -xeo pipefail \n\n""")
    
    fileNames = []
    for setting in ["All", "NoBB"]:
        fileNames.extend(glob.glob(os.path.join(root_folder, script_location, setting, "EN-P_EN-Q.jsonl")))       

    # manually add SQuAD
    fileNames.append(os.path.join(root_folder, "Data", "squad.jsonl"))
    
    for model_name in models:
        for fname in fileNames:
            output_dir_child = os.path.basename(fname).split('.')[0]
            template = f"""python {os.path.join(primeqa_path, "primeqa/mrc/run_mrc.py")} \
--model_name_or_path {model_name} \
--eval_file {fname} \
--preprocessor primeqa.mrc.processors.preprocessors.squad.SQUADPreprocessor \
--postprocessor primeqa.mrc.processors.postprocessors.squad.SQUADPostProcessor \
--eval_metrics SQUAD \
--output_dir {os.path.join(root_folder, output_dir_parent, "SQuAD_validation", output_dir_child)} \
--do_eval --per_device_eval_batch_size 8 \
--overwrite_output_dir --overwrite_cache"""
            shFile.write(template)
            shFile.write("\n\n")


## Generating Script for experiments in Table 3

### original/dialectal questions

In [ ]:
script_location = "Settings"
output_dir_parent = "Results"

for setting in ["All", "NoBB"]:
    print(f"Generating sh script for {setting} questions")
    for lang in ["EN", "MSA"]:
        print(f"{lang} Passages")
        fileNames = glob.glob(os.path.join(root_folder, script_location, setting, f"{lang}-P*"))

        write_script(root_folder, setting, lang, output_dir_parent, fileNames, primeqa_path, new_file=True)

### Translated dialectal questions

We translated dialectal questions to English, and to English->MSA 

In [ ]:
script_location = "Settings_Translated"
output_dir_parent = "Results_Translated"

for setting in ["All", "NoBB"]:
    print(f"Generating sh script for {setting} questions")
    for lang in ["EN", "MSA"]:
        print(f"{lang} Passages")
        fileNames = glob.glob(os.path.join(root_folder, script_location, setting, f"{lang}-P*"))

        write_script(root_folder, setting, lang, output_dir_parent, fileNames, primeqa_path, new_file=True, translated=True)

In [ ]:
# run each shell file from the files above

# now to notebook 3